In [2]:
__author__ = 'maoting'

import pandas as pd
import numpy as np
import statsmodels.api as sm
import pylab as pl
from scipy import stats
from sklearn.externals.six import StringIO
from sklearn import preprocessing
from sklearn import cluster, tree, decomposition
import matplotlib.pyplot as plt
import pydot

In [3]:
# import csv file
sample = pd.read_csv('sample')

In [4]:
# set display right
pd.set_option('display.width', 4000)
pd.set_option('max_colwidth', 4000)
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 200)
pd.set_option('float_format', '%.3f')

In [5]:
sample

,Unnamed: 0,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,24636210,2014-11-03 16:02:28,24,2,77,871,36643,456.115,792280,0,1,1,2014-12-15,2014-12-19,2,0,1,8286,1,0,1,0,63,1258,68
1,19837144,2013-03-13 19:25:01,11,3,205,135,38749,232.474,961995,0,0,9,2013-03-13,2013-03-14,2,0,1,1842,3,0,1,2,198,786,37
2,13066459,2014-10-13 13:20:25,2,3,66,314,48562,4468.272,495669,0,1,9,2015-04-03,2015-04-10,2,0,1,8746,1,0,1,6,105,29,22
3,4691082,2013-11-05 10:40:34,11,3,205,411,52752,171.602,106611,0,0,0,2013-11-07,2013-11-08,2,0,1,6210,3,1,1,2,198,1234,42
4,4878884,2014-06-10 13:34:56,2,3,66,174,50644,NaN,596177,0,0,9,2014-08-03,2014-08-08,2,1,1,12812,5,0,1,2,50,368,83
5,34825186,2014-12-16 14:34:39,2,3,66,220,38273,53.748,260938,0,0,9,2014-12-16,2014-12-17,2,0,1,3448,3,0,1,2,50,582,54
6,24023139,2014-01-28 13:16:36,2,3,66,337,54287,329.183,908558,0,0,2,2014-02-27,2014-03-01,2,0,1,8287,1,1,1,2,198,400,6
7,8184643,2014-02-27 08:29:19,24,2,3,38,4999,NaN,1161104,0,1,0,2014-07-16,2014-07-20,1,0,1,8856,1,0,1,2,50,636,0
8,19270014,2014-12-15 10:44:33,2,3,133,20,41185,NaN,560103,0,0,1,2014-12-17,2014-12-18,2,0,1,8226,1,0,1,5,100,1446,46
9,3997603,2013-11-16 23:40:35,2,3,66,189,17017,2284.910,748046,0,0,1,2014-01-10,2014-01-12,2,1,1,12206,6,0,1,2,50,628,1


In [6]:
#####################explore the data - descriptive statistics#####################
sample.head()
sample['user_location_city'].nunique()
sample.describe()
sample['date_time'].min()
sample['date_time'].max()
sample.info()
sample.describe()
sample.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
Unnamed: 0                   100000 non-null int64
date_time                    100000 non-null object
site_name                    100000 non-null int64
posa_continent               100000 non-null int64
user_location_country        100000 non-null int64
user_location_region         100000 non-null int64
user_location_city           100000 non-null int64
orig_destination_distance    63915 non-null float64
user_id                      100000 non-null int64
is_mobile                    100000 non-null int64
is_package                   100000 non-null int64
channel                      100000 non-null int64
srch_ci                      99878 non-null object
srch_co                      99878 non-null object
srch_adults_cnt              100000 non-null int64
srch_children_cnt            100000 non-null int64
srch_rm_cnt                  100000 non-null int64
srch_destination_id  

Unnamed: 0                     int64
date_time                     object
site_name                      int64
posa_continent                 int64
user_location_country          int64
user_location_region           int64
user_location_city             int64
orig_destination_distance    float64
user_id                        int64
is_mobile                      int64
is_package                     int64
channel                        int64
srch_ci                       object
srch_co                       object
srch_adults_cnt                int64
srch_children_cnt              int64
srch_rm_cnt                    int64
srch_destination_id            int64
srch_destination_type_id       int64
is_booking                     int64
cnt                            int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
hotel_cluster                  int64
dtype: object

In [7]:
# unique counts
def unique_counts(sample):
   for i in sample.columns:
       count = sample[i].nunique()
       print(i, ": ", count)

In [8]:
pd.crosstab(sample['is_booking'], sample['srch_rm_cnt'])

srch_rm_cnt,0,1,2,3,4,5,6,7,8
is_booking,,,,,,,,,
0,0,84425,6091,975,253,108,61,29,65
1,1,7248,580,102,42,9,6,1,4


In [9]:
sample.groupby('srch_rm_cnt')['is_booking'].mean()

srch_rm_cnt
0   1.000
1   0.079
2   0.087
3   0.095
4   0.142
5   0.077
6   0.090
7   0.033
8   0.058
Name: is_booking, dtype: float64

In [10]:
# correlations
pd.crosstab(sample['is_booking'], sample['srch_rm_cnt'])
sample.groupby('srch_rm_cnt')['is_booking'].mean()
sample['srch_children_cnt'].corr(sample['is_booking'])
sample.corr()

,Unnamed: 0,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
Unnamed: 0,1.000,0.005,-0.001,0.001,0.001,-0.002,-0.003,0.010,-0.006,-0.001,-0.000,-0.003,0.000,-0.004,0.001,-0.004,-0.003,-0.004,0.006,0.003,0.004,-0.006
site_name,0.005,1.000,-0.638,0.159,0.131,-0.013,0.028,0.030,-0.005,0.049,-0.028,-0.013,-0.032,0.017,0.035,-0.007,-0.013,0.022,0.202,0.263,-0.068,-0.027
posa_continent,-0.001,-0.638,1.000,0.180,-0.035,0.039,0.050,-0.015,0.016,-0.093,0.090,0.012,0.034,-0.034,-0.016,0.037,0.013,-0.019,-0.334,-0.157,0.049,0.018
user_location_country,0.001,0.159,0.180,1.000,0.058,0.123,0.048,-0.021,0.004,-0.025,0.110,0.043,0.037,0.001,0.013,0.029,0.001,0.004,-0.064,0.098,0.016,-0.012
user_location_region,0.001,0.131,-0.035,0.058,1.000,0.132,0.137,0.002,0.017,0.040,-0.002,0.005,0.014,0.000,0.023,0.001,0.000,-0.008,0.043,-0.050,0.040,0.005
user_location_city,-0.002,-0.013,0.039,0.123,0.132,1.000,0.014,-0.008,-0.004,0.013,0.023,0.007,0.003,-0.001,0.001,-0.004,-0.003,-0.002,0.008,-0.002,0.009,0.000
orig_destination_distance,-0.003,0.028,0.050,0.048,0.137,0.014,1.000,0.017,-0.059,0.042,-0.000,-0.024,-0.060,-0.012,-0.036,-0.043,-0.033,0.009,0.416,0.254,-0.090,0.004
user_id,0.010,0.030,-0.015,-0.021,0.002,-0.008,0.017,1.000,-0.011,-0.019,-0.004,-0.007,0.003,-0.002,0.003,0.007,0.002,0.001,0.002,0.009,-0.002,0.003
is_mobile,-0.006,-0.005,0.016,0.004,0.017,-0.004,-0.059,-0.011,1.000,0.047,-0.031,0.017,0.018,-0.023,-0.007,-0.016,-0.029,0.008,-0.024,-0.030,0.008,0.012
is_package,-0.001,0.049,-0.093,-0.025,0.040,0.013,0.042,-0.019,0.047,1.000,-0.011,-0.024,-0.038,-0.037,-0.147,-0.224,-0.081,0.126,0.109,-0.044,-0.015,0.031


In [11]:
# sample.hist()
sample[['channel', 'is_booking', 'is_mobile', 'orig_destination_distance', 'srch_rm_cnt', 'srch_adults_cnt', 'srch_children_cnt']].hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x10206fc88>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x10b659ac8>]], dtype=object)

In [12]:
# distribution of number of booking attempts
sample.groupby('user_id')['is_booking']\
   .agg({'num_of_bookings':'count'}).reset_index()\
   .groupby('num_of_bookings')['user_id']\
   .agg('count')

sample = sample.merge(sample.groupby('user_id')['is_booking']
    .agg(['count']).reset_index())

In [13]:
sample.groupby('user_id')['is_booking']\
   .agg(['mean']).reset_index()\
    .groupby('mean')['user_id']\
   .agg('count')

mean
0.000    80991
0.200        5
0.250       35
0.333      153
0.400        1
0.500     1054
0.600        1
0.667       26
0.750        1
1.000     6596
Name: user_id, dtype: int64

In [14]:
# distribution of booking rate
sample.groupby('user_id')['is_booking']\
   .agg(['mean']).reset_index()\
   .groupby('mean')['user_id']\
   .agg('count')

(sample.groupby('user_id')['is_booking']\
   .agg(['mean']).reset_index()\
   .groupby('mean')['user_id']\
   .agg('count').reset_index()).plot(x='mean', y='user_id')

In [15]:
#####################explore the data - validate data##################################

#number of guests need to be > 0
pd.crosstab(sample['srch_adults_cnt'], sample['srch_children_cnt'])
sample.drop(sample[sample['srch_adults_cnt'] + sample['srch_children_cnt']==0].index)

sample['srch_co'] = pd.to_datetime(sample['srch_co'])
sample['srch_ci'] = pd.to_datetime(sample['srch_ci'])
sample['date_time'] = pd.to_datetime(sample['date_time'])
sample['date'] = pd.to_datetime(sample['date_time'].apply(lambda x: x.date()))

In [16]:
# Check-out date need to be later than check-in date;
# Check-in date need to be later than booking date

sample[sample['srch_co'] < sample['srch_ci']][['srch_co', 'srch_ci']]
sample[sample['srch_ci'] < sample['date']][['srch_ci', 'date']]

,srch_ci,date
7138,2014-01-05,2014-01-07
14931,2014-10-13,2014-10-14
18778,2014-06-11,2014-11-10
22722,2014-02-11,2014-10-09
25886,2014-03-10,2014-09-15
26795,2014-09-12,2014-12-02
29591,2014-08-02,2014-08-23
33886,2013-01-16,2013-01-17
33887,2014-03-03,2014-03-04
33888,2013-01-21,2013-01-22


In [17]:
#####################explore the data - create new variables that might be useufl###########
def duration(row):
    delta = (row['srch_co'] - row['srch_ci'])/np.timedelta64(1, 'D')
    if delta <= 0:
        return np.nan
    else:
        return delta

def days_in_advance(row):
    delta = (row['srch_ci'] - row['date'])/np.timedelta64(1, 'D')
    if delta < 0:
        return np.nan
    else:
        return delta

sample['duration'] = sample.apply(duration, axis=1)
sample['days_in_advance'] = sample.apply(days_in_advance, axis=1)

In [18]:
############## Outperforming/underperforming segments #############
cat_list = ['site_name', 'posa_continent',
       'user_location_country', 'user_location_region',
       'user_location_city', 'channel',
       'srch_destination_id', 'srch_destination_type_id',
        'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster']

In [19]:
# for all columns
for i in cat_list:
    print(sample.groupby(i)['is_booking']
          .agg({'booking_rate': 'mean', 'num_of_bookings': 'sum'})
          .reset_index()
          .sort_values(by='booking_rate'))

    site_name  booking_rate  num_of_bookings
35         44         0.000                0
10         16         0.000                0
29         35         0.025                7
27         33         0.036               17
12         18         0.037               25
1           6         0.037                2
39         48         0.039                2
15         21         0.042                2
30         36         0.042                5
26         32         0.050               12
19         25         0.053               29
37         46         0.055                5
21         27         0.059               16
13         19         0.061                2
3           8         0.061              152
8          14         0.062                9
20         26         0.063               25
17         23         0.064              126
2           7         0.065                5
33         40         0.065                9
18         24         0.066              428
5         

In [20]:
stats.norm.cdf(1.2815515655446004)

0.89999999999999991

In [21]:
stats.norm.ppf(0.9)

1.2815515655446004

In [22]:
############## Outperforming/underperforming segments - two sample t test #############
def stats_comparison(i):
    sample.groupby(i)['is_booking'].agg({
    'average': 'mean',
    'bookings': 'count'
    }).reset_index()
    cat = sample.groupby(i)['is_booking']\
        .agg({
            'sub_average': 'mean',
            'sub_bookings': 'count'
       }).reset_index()
    cat['overall_average'] = sample['is_booking'].mean()
    cat['overall_bookings'] = sample['is_booking'].count()
    cat['rest_bookings'] = cat['overall_bookings'] - cat['sub_bookings']
    cat['rest_average'] = (cat['overall_bookings']*cat['overall_average'] \
                     - cat['sub_bookings']*cat['sub_average'])/cat['rest_bookings']
    cat['z_score'] = (cat['sub_average']-cat['rest_average'])/\
        np.sqrt(cat['overall_average']*(1-cat['overall_average'])
            *(1/cat['sub_bookings']+1/cat['rest_bookings']))
    cat['prob'] = np.around(stats.norm.cdf(cat.z_score), decimals = 10)
    cat['significant'] = [(lambda x: 1 if x > 0.9 else -1 if x < 0.1 else 0)(i) for i in cat['prob']]
    print(cat)

stats_comparison('user_location_city')

       user_location_city  sub_bookings  sub_average  overall_average  overall_bookings  rest_bookings  rest_average  z_score  prob  significant
0                       0             4        0.000             0.08            100000          99996          0.08   -0.589 0.278            0
1                       1             2        0.000             0.08            100000          99998          0.08   -0.417 0.338            0
2                       3            17        0.235             0.08            100000          99983          0.08    2.362 0.991            1
3                       7             1        0.000             0.08            100000          99999          0.08   -0.295 0.384            0
4                       8             1        0.000             0.08            100000          99999          0.08   -0.295 0.384            0
5                      14             4        0.000             0.08            100000          99996          0.08   -0.589 0.27

In [23]:
############## clustering - what are the similar user cities? ##############

# Step 1: what are the features I am going to use (that make sense)?
# What features may distinguish cities? based on business sense and exploratory analysis

num_list = ['duration', 'days_in_advance', 'orig_destination_distance', 'is_mobile',
            'is_package', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt']
city_data = sample.dropna(axis=0)[num_list + ['user_location_city']]
city_groups = city_data.groupby('user_location_city').mean().reset_index().dropna(axis=0)

In [24]:
# Step 2: shall I standardise the data?
# What is the magnitude of data range?

city_groups_std = city_groups.copy()
for i in num_list:
    city_groups_std[i] = preprocessing.scale(city_groups_std[i])

In [ ]:
# Step 3: select clustering method and number of clusters
# The Elbow methods? choose a K so that the sum of the square error of the distances decrease drastically
# using an ad-hoc k=3 here, there are methods to help derive the optimal number for k

km = cluster.KMeans(n_clusters=3, max_iter=300, random_state=None)
city_groups_std['cluster'] = km.fit_predict(city_groups_std[num_list])

# Principal Component Analysis
pca = decomposition.PCA(n_components=2, whiten=True)
pca.fit(city_groups[num_list])
city_groups_std['x'] = pca.fit_transform(city_groups_std[num_list])[:, 0]
city_groups_std['y'] = pca.fit_transform(city_groups_std[num_list])[:, 1]
plt.scatter(city_groups_std['x'], city_groups_std['y'], c=city_groups_std['cluster'])
plt.show()

In [25]:
# Step 4: profile the clusters
# merging the two dataframes based on a common column user_location_city
city_groups.merge(city_groups_std[['user_location_city', 'cluster']])\
    .groupby('cluster')\
    .mean() # for every column

KeyError: "['cluster'] not in index"

In [ ]:
# Step 5: assess the statistical robustness
# A statistically robust segmentation return similar results using different clustering methodologies

In [ ]:
############### decision tree - what lead to a higher chance of booking for individuals? ###############
from sklearn.cross_validation import train_test_split

In [26]:
# choose a cluster and split them into test and train
sample = sample.merge(city_groups_std[['user_location_city', 'cluster']], left_on='user_location_city', right_on='user_location_city', how='outer')
sample.groupby('cluster')['is_booking'].count()

KeyError: "['cluster'] not in index"

In [24]:
# choose one of the city clusters to analyze
tree_data = sample.dropna(axis = 0)[sample['cluster']==2]
tree_train, tree_test = train_test_split(tree_data, test_size=0.2, random_state=1, stratify=tree_data['is_booking'])

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [25]:
# build the decision tree model
clf = tree.DecisionTreeClassifier(max_leaf_nodes=6, min_samples_leaf=200)
clf = clf.fit(tree_train[num_list], tree_train['is_booking'])

In [26]:
# test_preds = clf.predict(X = tree_test[num_list])
# clf.predict_proba(tree_test[num_list])

In [27]:
# scoring of the prediction model
clf.score(tree_test[num_list], tree_test['is_booking'])

0.93627450980392157

In [28]:
# visualize the decision tree
dot_data = StringIO()
tree.export_graphviz(clf, out_file=dot_data, feature_names =['duration', 'days_in_advance', 'orig_destination_distance', 'is_mobile',
            'is_package', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt'], filled=True, rounded=True)
graph = pydot.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("booking_tree.pdf")

AttributeError: 'list' object has no attribute 'write_pdf'

In [ ]:
############### Logistic regression - what lead to a higher chance of booking for individuals? ###############
#create dummy variable channel
sample['channel'].value_counts()
sample['new_channel'] = [i if i in [9, 0, 1, 2] else 'other' for i in sample['channel']]
dummy_channels = pd.get_dummies(sample['new_channel'], prefix='channel')
sample = sample.join(dummy_channels.ix[:, :])
sample['log_orig_destination_distance'] = [np.log(i) for i in sample['orig_destination_distance']]

var_list = ['duration', 'days_in_advance', 'log_orig_destination_distance', 'is_mobile',
            'is_package', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'channel_1',
            'channel_2', 'channel_9', 'channel_other']

# create logistic estimator
logit = sm.Logit(sample['is_booking'], sample[var_list], missing='drop')
result = logit.fit()
result.summary()

# convert logit to odds ratio
np.exp(result.params)
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

# predicted values
sample['pred'] = result.predict(sample[var_list])

# plot variable VS. marketing_channel
sample.groupby('channel')['is_booking'].mean()

def plot_vs_marketing_channel(variable):
    grouped = pd.pivot_table(sample[(sample['pred'].isnull() == False)], values = ['pred'], index=[variable, 'new_channel'], aggfunc=np.mean)
    colors = 'rbgyrbgy'
    for col in sample.new_channel.unique():
        plt_data = grouped.ix[grouped.index.get_level_values(1)==col]
        pl.plot(plt_data.index.get_level_values(0), plt_data['pred'])
    pl.xlabel(variable)
    pl.ylabel('prob booking = 1')
    pl.legend(['channel_'+str(i) for i in list(sample.new_channel.unique())], loc='upper right', title='new channel')
    # pl.show()
plot_vs_marketing_channel('days_in_advance')